In [33]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [2]:
%run data.ipynb

In [3]:
%run processing_methods.ipynb

In [4]:
%run ml_methods.ipynb

In [5]:
x_ = x_all.copy()
y_ = y_all.copy()

In [6]:
x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)

In [64]:
data_sets = []
best_scores = []
test_scores = []
results = []
models = []
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
    data_sets.append([x_trainval, x_test, y_trainval, y_test, x_train, x_val, y_train, y_val])
    
    best_score = 0
    for C in [0.01, 0.1, 1, 5, 10, 50, 100, 200, 300, 400, 500]:
        
        model = LogisticRegression(C=C, max_iter=100000).fit(x_train, y_train)
        score = model.score(x_val, y_val)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C}

    model  = LogisticRegression(C=best_parameters['C'], max_iter=100000).fit(x_trainval, y_trainval)
    test_score = model.score(x_test, y_test)
    results.append(best_parameters)
    test_scores.append(test_score)
    best_scores.append(best_score)
    models.append(model)

In [65]:
for i in range(20):
    print("{}: ".format(i)+"best_parametrs: {}, ".format(results[i])+"test_score: {}, ".format(test_scores[i])+"best_score: {}, ".format(best_scores[i]))

0: best_parametrs: {'C': 400}, test_score: 0.8888888888888888, best_score: 0.8992248062015504, 
1: best_parametrs: {'C': 500}, test_score: 0.8654970760233918, best_score: 0.8682170542635659, 
2: best_parametrs: {'C': 100}, test_score: 0.8830409356725146, best_score: 0.8914728682170543, 
3: best_parametrs: {'C': 300}, test_score: 0.9005847953216374, best_score: 0.9534883720930233, 
4: best_parametrs: {'C': 500}, test_score: 0.9064327485380117, best_score: 0.875968992248062, 
5: best_parametrs: {'C': 500}, test_score: 0.8947368421052632, best_score: 0.8992248062015504, 
6: best_parametrs: {'C': 200}, test_score: 0.8888888888888888, best_score: 0.8837209302325582, 
7: best_parametrs: {'C': 200}, test_score: 0.9064327485380117, best_score: 0.8992248062015504, 
8: best_parametrs: {'C': 400}, test_score: 0.935672514619883, best_score: 0.8992248062015504, 
9: best_parametrs: {'C': 50}, test_score: 0.9122807017543859, best_score: 0.8992248062015504, 
10: best_parametrs: {'C': 100}, test_score:

In [90]:
c_col = []
for i in range(20):
    c_col.append(results[i]['C'])
c_col = pd.Series(np.array(c_col))
test_col = pd.Series(test_scores)
best_col = pd.Series(best_scores)
res = pd.concat([c_col,best_col,test_col],axis=1)
res.to_excel('results/logistic_general.xlsx')

In [91]:
mean = test_col.mean()
print("Mean: {},".format(mean))

Mean: 0.9055555555555556,


In [92]:
mode = c_col.mode()
mode[0]

1000

In [ ]:
mode = c_col.mode()
mode[0]

In [69]:
# best test score
iter = test_col.idxmax()
y_pred = models[iter].predict(data_sets[iter][1])
y_test = data_sets[iter][3]
classification_report_ = classification_report(y_test,y_pred,output_dict=True)
classification_report_ = pd.DataFrame(classification_report_)
classification_report_

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.943262,0.900000,1.000000,0.94152,0.947754,0.942673
recall,0.992537,0.818182,0.666667,0.94152,0.825795,0.941520
f1-score,0.967273,0.857143,0.800000,0.94152,0.874805,0.938431
support,134.000000,22.000000,15.000000,0.94152,171.000000,171.000000


In [70]:
confusion_matrix_ = confusion_matrix(y_test,y_pred)
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,133,1,0
1,4,18,0
2,4,1,10


In [71]:
# worst test score
iter = test_col.idxmin()
y_pred = models[iter].predict(data_sets[iter][1])
y_test = data_sets[iter][3]
classification_report_ = classification_report(y_test,y_pred,output_dict=True)
classification_report_ = pd.DataFrame(classification_report_)
classification_report_

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.878378,0.722222,0.400000,0.847953,0.666867,0.813528
recall,0.977444,0.590909,0.125000,0.847953,0.564451,0.847953
f1-score,0.925267,0.650000,0.190476,0.847953,0.588581,0.821100
support,133.000000,22.000000,16.000000,0.847953,171.000000,171.000000


In [72]:
confusion_matrix_ = confusion_matrix(y_test,y_pred)
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,130,2,1
1,7,13,2
2,11,3,2


{'C': 50}

In [77]:
scores_C = []
results_C = []
y_pred = 0
best_score = 0
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
      
    model  = LogisticRegression(C=mode[0], max_iter=100000).fit(x_trainval, y_trainval)
    score = model.score(x_test, y_test)
    if score > best_score:
        y_pred = model.predict(x_test)
        confusion_matrix_ = confusion_matrix(y_test,y_pred)
        classification_report_ = classification_report(y_test,y_pred,output_dict=True)
    scores_C.append(score)


In [79]:
for i in range(20):
    print("{}: ".format(i)+"C: {}, ".format(132)+"test_score: {}, ".format(scores_C[i]))

0: C: 132, test_score: 0.8888888888888888, 
1: C: 132, test_score: 0.8888888888888888, 
2: C: 132, test_score: 0.8888888888888888, 
3: C: 132, test_score: 0.9122807017543859, 
4: C: 132, test_score: 0.8713450292397661, 
5: C: 132, test_score: 0.8771929824561403, 
6: C: 132, test_score: 0.9239766081871345, 
7: C: 132, test_score: 0.9122807017543859, 
8: C: 132, test_score: 0.9181286549707602, 
9: C: 132, test_score: 0.8947368421052632, 
10: C: 132, test_score: 0.9181286549707602, 
11: C: 132, test_score: 0.9122807017543859, 
12: C: 132, test_score: 0.8830409356725146, 
13: C: 132, test_score: 0.9298245614035088, 
14: C: 132, test_score: 0.9181286549707602, 
15: C: 132, test_score: 0.9298245614035088, 
16: C: 132, test_score: 0.8713450292397661, 
17: C: 132, test_score: 0.9298245614035088, 
18: C: 132, test_score: 0.9064327485380117, 
19: C: 132, test_score: 0.9239766081871345, 


In [80]:
scores_C = pd.Series(scores_C)
res_C = pd.concat([scores_C],axis=1)
res_C.to_excel('results/logistic_general_best_C.xlsx')

In [81]:
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,119,3,4
1,3,27,0
2,3,0,12


In [85]:
classification_report_df = pd.DataFrame(classification_report_)
classification_report_df

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.952000,0.9,0.750000,0.923977,0.867333,0.925158
recall,0.944444,0.9,0.800000,0.923977,0.881481,0.923977
f1-score,0.948207,0.9,0.774194,0.923977,0.874134,0.924485
support,126.000000,30.0,15.000000,0.923977,171.000000,171.000000


In [80]:
y_gpa_ = pd.Series(y_gpa)
x_gpa_ = pd.DataFrame(x_gpa)
x_gpa_

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.015816,0.113986,-0.017357,-0.004689,0.091246,0.067890,0.150166,-0.008785,0.033184,-0.112502,...,-0.921157,-0.90,-0.4,-1.000000,-1.00,-0.932243,-0.770245,-0.574553,-1.0,-1.0
1,0.070616,0.002053,-0.062461,0.075655,0.160422,-0.005790,-0.018588,-0.007662,-0.016452,-0.003504,...,-0.754045,-0.75,-0.2,-1.000000,-1.00,-0.602804,-0.973635,-0.495030,-1.0,-1.0
2,0.062143,0.019626,0.023620,0.025012,0.056093,-0.045252,0.053993,0.055094,0.005608,-0.032532,...,-0.621212,-0.70,-0.8,-0.746624,-0.98,-0.268692,-0.996234,-0.741551,-1.0,-1.0
3,0.070944,-0.007633,-0.082040,0.034312,0.018154,-0.011309,-0.037304,-0.014980,-0.060727,0.035697,...,-0.924242,-1.00,-1.0,-1.000000,-1.00,0.317757,-0.962335,-0.610338,-1.0,-1.0
4,0.061455,-0.020190,-0.026374,-0.057199,-0.050706,0.015966,-0.019512,0.040170,-0.028181,-0.031372,...,-0.939394,-1.00,-1.0,-0.678457,-1.00,-0.242991,-0.984934,-0.662028,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0.081600,0.008834,0.124497,-0.066554,-0.035240,-0.030343,0.017054,-0.018411,0.029497,0.008461,...,-0.924242,-1.00,-1.0,-1.000000,-1.00,-0.032710,-1.000000,-0.705765,-1.0,-1.0
547,0.070916,0.021262,0.073997,-0.053416,-0.022171,-0.052697,0.059967,-0.046477,0.032523,0.015784,...,-0.924242,-1.00,-1.0,-1.000000,-1.00,-0.785047,-0.939736,-1.000000,-1.0,-1.0
548,0.093099,0.096175,0.144220,0.015833,-0.102907,-0.003576,0.113400,-0.039033,-0.042474,-0.007542,...,-0.939394,-1.00,-1.0,-1.000000,-1.00,-0.324766,-1.000000,0.065606,-1.0,-1.0
549,-3.206259,0.899733,-0.068907,-0.198480,0.108945,-0.440711,-0.452796,0.112308,0.189137,-0.098345,...,-1.000000,-1.00,-1.0,-0.742765,-0.98,0.067757,-1.000000,-0.685885,-1.0,-1.0


In [83]:
data_sets = []
best_scores = []
test_scores = []
results = []
models = []
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_gpa, y_gpa)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
    data_sets.append([x_trainval, x_test, y_trainval, y_test, x_train, x_val, y_train, y_val])
    
    best_score = 0
    for C in [ 0.01, 0.1, 1, 5, 10,20,30,  40,  50, 60, 70, 100, 200]:
        
        model = LogisticRegression(C=C, max_iter=100000).fit(x_train, y_train)
        score = model.score(x_val, y_val)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C}

    model  = LogisticRegression(C=C, max_iter=1000).fit(x_trainval, y_trainval)
    test_score = model.score(x_test, y_test)
    results.append(best_parameters)
    test_scores.append(test_score)
    best_scores.append(best_score)
    models.append(model)

In [ ]:
for i in range(20):
    print("{}: ".format(i)+"best_parametrs: {}, ".format(results[i])+"test_score: {}, ".format(test_scores[i])+"best_score: {}, ".format(best_scores[i]))

In [ ]:
c_col = []
for i in range(20):
    c_col.append(results[i]['C'])
c_col = pd.Series(np.array(c_col))
test_col = pd.Series(test_scores)
best_col = pd.Series(best_scores)
res = pd.concat([c_col,best_col,test_col],axis=1)
res.to_excel('results/logistic_general.xlsx')

In [ ]:
# best test score
iter = 0
y_pred = models[iter].predict(data_sets[iter][1])
y_test = data_sets[iter][3]
classification_report_ = classification_report(y_test,y_pred,output_dict=True)
classification_report_ = pd.DataFrame(classification_report_)
classification_report_

In [ ]:
confusion_matrix_ = confusion_matrix(y_test,y_pred)
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

In [86]:
data_sets = []
best_scores = []
test_scores = []
results = []
models = []
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
    data_sets.append([x_trainval, x_test, y_trainval, y_test, x_train, x_val, y_train, y_val])
    
    best_score = 0
    for C in [0.01, 0.1, 1, 5, 10, 50, 100, 200, 300, 400, 500, 1000]:
        
        model = LogisticRegression(C=C, max_iter=100000).fit(x_train, y_train)
        score = model.score(x_val, y_val)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C}

    model  = LogisticRegression(C=best_parameters['C'], max_iter=100000).fit(x_trainval, y_trainval)
    test_score = model.score(x_test, y_test)
    results.append(best_parameters)
    test_scores.append(test_score)
    best_scores.append(best_score)
    models.append(model)

In [87]:
for i in range(20):
    print("{}: ".format(i)+"best_parametrs: {}, ".format(results[i])+"test_score: {}, ".format(test_scores[i])+"best_score: {}, ".format(best_scores[i]))

0: best_parametrs: {'C': 50}, test_score: 0.8888888888888888, best_score: 0.875968992248062, 
1: best_parametrs: {'C': 200}, test_score: 0.9122807017543859, best_score: 0.8992248062015504, 
2: best_parametrs: {'C': 1000}, test_score: 0.9181286549707602, best_score: 0.9457364341085271, 
3: best_parametrs: {'C': 200}, test_score: 0.8596491228070176, best_score: 0.937984496124031, 
4: best_parametrs: {'C': 100}, test_score: 0.9005847953216374, best_score: 0.9224806201550387, 
5: best_parametrs: {'C': 200}, test_score: 0.8830409356725146, best_score: 0.8837209302325582, 
6: best_parametrs: {'C': 300}, test_score: 0.9649122807017544, best_score: 0.875968992248062, 
7: best_parametrs: {'C': 1000}, test_score: 0.9298245614035088, best_score: 0.8837209302325582, 
8: best_parametrs: {'C': 500}, test_score: 0.9239766081871345, best_score: 0.9302325581395349, 
9: best_parametrs: {'C': 1000}, test_score: 0.9181286549707602, best_score: 0.9069767441860465, 
10: best_parametrs: {'C': 200}, test_scor

In [88]:
mean = test_col.mean()
print("Mean: {},".format(mean))

Mean: 0.9038011695906434,
